<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/Cosine_Similarity_Recommender/MovieLens_CS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> Cosine similarity Recommender</font></h1>

---

<center>
  <img src="https://bobliu.io/assets/img/cards.509a5045.jpg" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en Cosine Similarity.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

## 1. Contexto


El conjunto de datos MovieLens es uno de los conjuntos de datos de recomendación más populares y ampliamente utilizados en la investigación de sistemas de recomendación. Fue creado por el GroupLens Research Project en la Universidad de Minnesota para impulsar la investigación en sistemas de recomendación, proporcionando un recurso valioso para la comunidad académica y promoviendo el desarrollo y la comprensión de tecnologías de recomendación personalizada.


<b>Descripción de datos</b>

El conjunto de datos MovieLens contiene información sobre:

<b>Películas:</b> Detalles sobre las películas, incluyendo su título, género y año de lanzamiento.

<b>Usuarios:</b> Perfiles de los usuarios que han calificado y/o etiquetado las películas, incluyendo su ID y otros detalles demográficos opcionales.

<b>Calificaciones:</b> Calificaciones numéricas que los usuarios asignan a las películas en una escala de 1 a 5.

<b>Etiquetas:</b> Palabras clave o tags proporcionados por los usuarios para describir el contenido o la esencia de las películas.

El conjunto de datos es ampliamente utilizado con fines académicos y de investigación, siendo una referencia en el diseño y evaluación de sistemas de recomendación de películas. También es útil para el análisis de tendencias y comportamientos en la visualización de películas y la interacción del usuario con el contenido.

<strong>Puede consultar este [link](https://grouplens.org/datasets/movielens/) para leer más sobre la fuente de datos MovieLens proporcionada por GroupLens Research en la Universidad de Minnesota.</strong>

## 2. Descargar y preparar Dataset

In [ ]:
# Descargar el dataset Movielens
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
!unzip dataset.zip
!ls -la

In [3]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [4]:
links   = pd.read_csv("ml-latest-small/links.csv")
movies  = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
tags    = pd.read_csv("ml-latest-small/tags.csv")


In [ ]:
links.head()

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
tags.head()

In [ ]:
print("  Movies: {} \n  Ratings: {}".format(len(movies), len(ratings)))


In [10]:
# Fusiona ambos datasets basados en la columna 'movieId'
data = pd.merge(ratings, movies, on='movieId')

In [11]:
movie_titles = data['title'].unique().tolist()
movie_ids = data['movieId'].unique().tolist()


In [12]:
# Crear matriz pivotada de usuarios y películas
user_movie_rating = data.pivot_table(index='userId', columns='title', values='rating').fillna(0)


Muestreo

In [13]:
from sklearn.model_selection import train_test_split

# Convertir la matriz pivotada en un DataFrame y dividir en train y test
ratings_df = user_movie_rating.stack().reset_index(name='rating')
train_data, test_data = train_test_split(ratings_df, test_size=0.2, random_state=42)


In [14]:
# Volver a crear matrices pivotadas para entrenamiento y prueba
train_data_matrix = train_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)
test_data_matrix = test_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)

Tratamiento de missings

In [15]:
# Llenar los NaN con ceros
train_data_matrix = train_data_matrix.fillna(0)


## 3. Cosine Similarity

Aplicaremos el enfoque de similaridad de coseno con enfoque ítem-ítem

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similaridad de coseno entre películas en el conjunto de entrenamiento
movie_similarity = cosine_similarity(train_data_matrix.T)
# Ojo, acá esta el detalle importante porq cosine_similarity calcula la similaridad de las filas, entonces de eso depende como orientamos la estrategia de similaridad

In [ ]:
# Seleccionar un usuario (por ejemplo, el usuario con ID 5)
user_id = 5

# Películas que el usuario ha visto
user_movies = data[data['userId'] == user_id]
watched_movies = user_movies[['title', 'genres', 'rating']]
watched_movies.sort_values(by = 'rating', ascending = False, inplace = True)

print(f"Películas vistas por el usuario {user_id}:")
watched_movies.head(10)

In [ ]:
# Utilizar el modelo de recomendación para obtener recomendaciones

# la matriz de usuario-ítem (user_movie_matrix)
user_vector = train_data_matrix.loc[user_id].values

# Calcular el puntaje de todas las películas para este usuario
scores = np.dot(user_vector, movie_similarity)

# Obtener índices de las películas ordenadas por puntaje
movie_indices = scores.argsort()[::-1]

# Filtrar las películas que el usuario ya ha visto
recommended_indices = [index for index in movie_indices if index not in watched_movies.index][:10]

recommended_movies = data[['title', 'genres']].iloc[recommended_indices]

print(f"\nRecomendaciones para el usuario {user_id}:")
recommended_movies.head(5)

In [14]:
%%capture
!pip install adjustText

PCA para entender la similaridad

In [ ]:
from sklearn.decomposition import PCA
from adjustText import adjust_text

# Aplicar PCA
pca = PCA(n_components=2)
movie_pca = pca.fit_transform(movie_similarity)

# Obtener las 30 películas más vistas
movie_view_counts = data.groupby('title').size().reset_index(name='view_count')
top_30_movies = movie_view_counts.sort_values('view_count', ascending=False).head(30)
top_movie_indices = [movie_titles.index(title) for title in top_30_movies['title']]
top_movie_pca = movie_pca[top_movie_indices]
top_movie_titles = [movie_titles[i] for i in top_movie_indices]

# Graficar las 30 películas más vistas usando PCA
plt.figure(figsize=(20, 10))
plt.scatter(top_movie_pca[:, 0], top_movie_pca[:, 1], s=50)

texts = []
for i in range(30):
    texts.append(plt.text(top_movie_pca[i, 0], top_movie_pca[i, 1], top_movie_titles[i], fontsize=7))

# Evitar superposición de etiquetas
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Visualización de la Similaridad de las 30 Películas Más Vistas usando PCA')
plt.show()

TSNE para entender la similaridad

In [ ]:
from sklearn.manifold import TSNE
from adjustText import adjust_text

# Aplicar t-SNE
tsne = TSNE(n_components=2, random_state=123)
movie_tsne = tsne.fit_transform(movie_similarity)

# Obtener las 30 películas más vistas
movie_view_counts = data.groupby('title').size().reset_index(name='view_count')
top_30_movies = movie_view_counts.sort_values('view_count', ascending=False).head(30)
top_movie_indices = [movie_titles.index(title) for title in top_30_movies['title']]
top_movie_tsne = movie_tsne[top_movie_indices]
top_movie_titles = [movie_titles[i] for i in top_movie_indices]

# Graficar las 30 películas más vistas usando t-SNE
plt.figure(figsize=(20, 10))
plt.scatter(top_movie_tsne[:, 0], top_movie_tsne[:, 1], s=50)

texts = []
for i in range(30):
    texts.append(plt.text(top_movie_tsne[i, 0], top_movie_tsne[i, 1], top_movie_titles[i], fontsize=7))

# Evitar superposición de etiquetas
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('Visualización de la Similaridad de las 30 Películas Más Vistas usando t-SNE')
plt.show()

## 4. Evaluación del modelo

Evaluación del hit Rate

In [36]:
def predict_top_n_movies_for_user(user_vector, item_similarity, n=10):
    """Predecir las n películas recomendadas para un usuario."""
    predicted_scores = user_vector.dot(item_similarity)
    # Obtener índices de los ítems con las puntuaciones más altas
    recommended_item_indices = np.argsort(predicted_scores)[::-1][:n]
    return recommended_item_indices

def hit_rate(user_matrix, item_similarity, test_matrix, n=10):
    """Calcular el hit rate del modelo."""
    hits = 0
    total = 0

    for user_idx in range(user_matrix.shape[0]):
        test_movies = np.where(test_matrix[user_idx] > 0)[0]  # Ítems con los que el usuario interactuó en el conjunto de prueba
        if len(test_movies) == 0:
            continue  # Si el usuario no tiene ítems en el conjunto de prueba, saltar al siguiente usuario
        top_n_predicted = predict_top_n_movies_for_user(user_matrix[user_idx], item_similarity, n)
        if set(top_n_predicted) & set(test_movies):
            hits += 1
        total += 1

    return hits / total if total != 0 else 0

In [ ]:
# Calculando hit rate para test
test_hit_rate = hit_rate(train_data_matrix.values, movie_similarity, test_data_matrix.values)
print(f"Hit Rate (Test): {test_hit_rate:.4f}")

MAE, RMSE

In [46]:
from sklearn.metrics import *

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return np.sqrt(mean_squared_error(prediction, ground_truth))

def mae(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return mean_absolute_error(prediction, ground_truth)

In [ ]:
train_rmse = rmse(movie_similarity, train_data_matrix.values)
test_rmse = rmse(movie_similarity, test_data_matrix.values)

print(f"RMSE (Train): {train_rmse:.4f}")
print(f"RMSE (Test): {test_rmse:.4f}")

In [ ]:
train_mae = mae(movie_similarity, train_data_matrix.values)
test_mae = mae(movie_similarity, test_data_matrix.values)

print(f"MAE (Train): {train_mae:.4f}")
print(f"MAE (Test): {test_mae:.4f}")

---
## Gracias por completar este laboratorio!